## 日本語らしさを計算する

文の中の単語をランダムに並び替えて、言語モデルを使って「どの程度日本語らしいか」を計算してみる。

### 言語モデル関数の定義

In [1]:
from nltk.lm import Vocabulary
from nltk.lm.models import MLE
# MLE(Maximum Likelihood Estimator):言語モデルを作成するライブラリ
from nltk.util import ngrams

# 文内の単語のを取得するための関数を作成
def find_xs_in_y(xs, y):
    return [x for x in xs if y['begin'] <= x['begin'] and x['end'] <= y['end']]
'''
リスト内包表記
for x in xs:
    if y['begin'] <= x['begin'] and x['end'] <= y['end']:
        return x

xs変数に格納されているアノテーションのリストからyアノテーションの内側に存在するものだけを取り出す関数
'''

#言語モデルの作成
def create_language_model(doc_ids, N=3):
    sents = []
    
    # コーパスとして文ごとに単語の原型のリストをsents変数に格納する
    for doc_id in doc_ids:
        all_tokens = datastore.get_annotation(doc_id, 'token')
        
        for sent in datastore.get_annotation(doc_id, 'sentence'):
            tokens = find_xs_in_y(all_tokens, sent)
            # コーパスとして文ごとに単語の原型リストをsents変数に格納する。文頭と文末も単語として追加する。
            sents.append(['__BOS__'] + [token['lemma'] for token in tokens] + ['__EOS__'])
            
    # 作成したコーパスにおける全単語の一覧をvocabに作成
    vocab = Vocabulary([word for sent in sents for word in sent])
    # ngramsを使用し文ごとに3つの単語の並びを取り出す
    text_ngrams = [ngrams(sent, N) for sent in sents]
    '''
    ngramの第一引数には単語リスト、第2引数には関数で指定したデフォルトの3を入れる
    '''
    lm = MLE(order=N, vocabulary=vocab)
    lm.fit(text_ngrams)
    return lm

### 日本語らしさの確率を計算する関数を定義

In [2]:
# (算出した言語モデル, 単語列, n-gramのn)
def calc_prob(lm, lemmas, N=3):
    probability = 1.0
    
    # ngramsを使用しN=3の単語の並びを作成し、一つずつ言語モデルを計算する。
    for ngram in ngrams(lemmas, N):
        prob = lm.score(lm.vocab.lookup(ngram[-1]), lm.vocab.lookup(ngram[:-1]))
        # 学習したコーパスにない単語に関しては並びの確率に10の-8乗を割り当てる。
        prob = max(prob, 1e-8)
        #  計算した単語の並びの確率をかけ合わせて、文の確率を計算する。
        probability *= prob
    return probability

### 計算開始

#### ライブラリの読み込み

In [3]:
# https://www.sejuku.net/blog/66459
# システムに関する処理をまとめたライブラリのsysを読み込む
import sys
# 下記でライブラリを読み込めるパス一覧を表示できる。ここにパスを書き込むと異なる階層からライブラリを読み込む事が可能となる。
print(sys.path)
# sys.path.append("相対パス")でsys.pathに追加、ここではディレクトリまでを指定する
sys.path.append("src")

import sqlitedatastore as datastore
import cabochaparser as parser

import random

['/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '', '/home/vagrant/.local/lib/python3.6/site-packages', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/home/vagrant/.local/lib/python3.6/site-packages/IPython/extensions', '/home/vagrant/.ipython']


In [5]:
datastore.connect()

# 全てのidで言語モデルを作成
lm = create_language_model(datastore.get_all_ids(limit=-1), N=3)

# テストする文を設定
text = '古くから人が居住する。'

# cabochaを実行し、textをtokenに分割する
sentences, chunks, tokens = parser.parse(text)

probabilities = set([])
for i in range(1000):
    # 2語目以降のtokenをシャッフルしtokens_に収納
    tokens_ = tokens[1:]
    random.shuffle(tokens_)
    # 1語目とくっつける
    tokens_shuffled = [tokens[0]] + tokens_
    lemmas = ['__BOS__'] + [token['lemma'] for token in tokens_shuffled] + ['__EOS__']
    # 文ごとに改行を加える。
    shuffled_text = ''.join([text[token['begin']:token['end']] for token in tokens_shuffled])
    # 言語モデルを実行
    probability = calc_prob(lm, lemmas, N=3)
    # 結果をprobabilitiesに格納
    probabilities.add((probability, shuffled_text))

# probabilityの高い順に並び替え上位20件を取得する
for probability, shuffled_text in sorted(list(probabilities), reverse=True)[:20]:
    print(f'{probability:e}: {shuffled_text:s}')
    
datastore.close()

8.112896e-06: 古くから人が居住する
3.392130e-19: 古くから居住する人が
1.492537e-26: 古くから居住するが人
7.462687e-27: 古くから人がする居住
2.597403e-27: 古くからする人が居住
4.782609e-33: 古くからが居住する人
4.972376e-34: 古く人がから居住する
4.972376e-34: 古くが人から居住する
4.756185e-34: 古く人からが居住する
2.486188e-34: 古くからが人居住する
2.285714e-34: 古くするから人が居住
2.186335e-34: 古く人が居住するから
1.492537e-34: 古くから居住人するが
1.492537e-34: 古くから居住人がする
1.492537e-34: 古くから居住が人する
1.492537e-34: 古くから居住がする人
7.462687e-35: 古くから人居住するが
7.462687e-35: 古くから人居住がする
7.462687e-35: 古くから人する居住が
7.462687e-35: 古くから人するが居住
